2019-02-04: INSeq Klebsiella Report
===================================

## Purpose

* This notebook will contain an overview of the analysis done on INSeq data from a colonization experiment of *Klebsiella Pneumonia*

### Computational Experiments:

1. Pyinseq - gave a table with hits per gene in every sample, this was used in analysis
2. DESeq Statistical Analysis - did statistical test taking into consideration the behavior of count data produced from high-throughput sequencing experiments.

### Import summary gene table data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure, output_file
from bokeh.models import *
from bokeh.layouts import column, row, gridplot

import ipywidgets as widgets
from ipywidgets import interact, interact_manual, widgets
output_notebook()

Loading BokehJS ...

* Summary gene table contains the output results from the pyinseq run. 

In [2]:
summary_df = pd.read_csv("2019-01-31_E1004_renamed.txt", sep='\t')
summary_df.head(1)

,Contig,Start,End,Strand,Length,PID,Gene,Synonym,Code,COG,Product,ket/xyl_E1004_01,ket/xyl_E1004_03,ket/xyl_E1004_04,propofol_E1004_05,propofol_E1004_06,propofol_E1004_08,input_E1004_13,input_E1004_14,input_E1004_15
0,CP009208,15,227,+,213,AIK78632.1,-,VK055_0001,-,-,lambda phage tail tape-measure family protein,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Sample correlation was performed using **spearman correlation
** Less sensitive to outliers

* As expected,

<img src="2019-01-31_E1004_spearman_heatmap.png" width=800>

### And a T50 Test
Counts how many hits made up for than 50% of all hits in the sample. Is used to show if there is a bottleneck in the INSeq experiment

In [3]:
with open("experiments/2018-10-26_pyinseq-runs/klebsiella-disruption_09/log.txt") as log:
    nfifty_d = {}
    for line in log:
        if "N50" in line.split(" "):
            line = line.split(" ")
            if len(line) == 10:
                pass
            else:
                sample = line[10].replace(":","")
                n50 = line[11].replace("\n", "")
                nfifty_d[sample] = int(n50)

fig_T50, ax_T50 = plt.subplots()
fig_T50.set_size_inches(10,8)

# Set x labels
ax_T50.set_xticks(range(len(nfifty_d)))
ax_T50.set_xticklabels(list(nfifty_d.keys()))

ax_T50.bar(range(len(nfifty_d)), list(nfifty_d.values()), align='center')

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/2018-10-26_pyinseq-runs/klebsiella-disruption_09/log.txt'

##### Conclusion
* No significant bottleneck was observed

### DESeq statistical analysis with Fold-Change and pvalue calculations
* After samples were processed by Pyinseq and hits were mapped to correspondent gene, the count table was submitted to the DESeq pipeline. Even though DESeq was orginally meant for studying differentially expressed genes by using RNA-Seq, the statistical tools it uses can be easily implemented for INSeq experiments since the data behaves the same way. This way we can normalize the data to avoid any bias by low-count genes and errors in the sequencing process. 

It outputs a p-adjusted value which uses Benjamini-Hochberg procedure to recude false true discoveries (Type-1 Errors). Also includes log2FC using input as the denominator of the ratio.

In [ ]:
deseq_df = pd.read_csv("2019-01-31_deseq_table_results.txt", sep='\t')
columns = list(deseq_df.columns[2:])
deseq_df.set_index("Synonym").sort_values("Ketamine log10FC").head()

### Previously selected genes
* Previous genes were selected based on the fold-change regarding the input count values. However, after running both Mann-Whitney test and a Wald T-Test, from DESeq, many were shown to be statistically non-significant.

In [ ]:
previous_genes = [gene.replace("\n","") for gene in open("genesOfInterest.txt")]
previous_genes = deseq_df.set_index("Synonym").loc[previous_genes]

fig_ella = plt.figure(figsize=(12,10), constrained_layout=True)
grid_ella = mpl.gridspec.GridSpec(3,4, figure=fig_ella)


r = 0
c = 0
for i in range(len(previous_genes)):
    ax = fig_ella.add_subplot(grid_ella[r,c])
    if c == 3:
        r += 1
        c = 0
    else:
        c += 1
    
    gene = {
        "Input": list(previous_genes.iloc[i,16:]),
        "Propofol": list(previous_genes.iloc[i,13:16]),
        "Ketamine": list(previous_genes.iloc[i,10:13])
    }
    
    ax.boxplot(list(gene.values()), labels=list(gene.keys()), whis=10)
    ax.set_title(f"{previous_genes.iloc[i,:].Gene} ({previous_genes.iloc[i,:].name})")
previous_genes

#### Volcano Plot
* To visualize this, I have set up a volcano plot highlighting the previous genes of interest. There is a hover tool available which allows you to see metadata of that data point, try it. In addition, you can hide data points by toggling the legend name on the right legend panel.

In [ ]:
# Make DataSources
previous_source = ColumnDataSource(previous_genes)
all_source = ColumnDataSource(deseq_df)

# Trend Lines
vline = Span(location=np.log10(1/4), dimension='height', line_color='black', line_width=1)
hline = Span(location=0.05, dimension='width', line_color='black', line_width=1)

# Initiate figures
fig_propofol = figure(plot_height=400, plot_width=400, title="Propofol Volcano Plot", 
            tools="crosshair,pan,reset, save,wheel_zoom, lasso_select, hover",
            x_range=[-5,5], y_range=[10**0,0.001], tooltips=[
    ("P-Value adj", "@{Propofol padj}"),
    ("Synonym","@Synonym"),
    ("Gene_ID", "@Gene"),
    ("Fold-Change", "@{Propofol log10FC}")
], y_axis_type="log")

fig_ketamine = figure(plot_height=400, plot_width=400, title="Ketamine Volcano Plot", 
            tools="crosshair,pan,reset, save,wheel_zoom, lasso_select, hover",
            x_range=[-5,5], y_range=[10**0,0.001], tooltips=[
    ("P-Value adj", "@{Ketamine padj}"),
    ("Synonym","@Synonym"),
    ("Gene_ID", "@Gene"),
    ("Fold-Change", "@{Ketamine log10FC}")
], y_axis_type="log")

figures = [fig_ketamine,fig_propofol]

fig_propofol.scatter(x="Propofol log10FC", y="Propofol padj", source=all_source, fill_color='gray', line_color="gray", size=4, legend="All Genes")
fig_propofol.scatter(x="Propofol log10FC", y="Propofol padj", source=previous_source, fill_color="red", line_color="red", size=8, legend="Previous Genes")

fig_ketamine.scatter(x="Ketamine log10FC", y="Ketamine padj", source=all_source, fill_color='gray', line_color="gray", size=4, legend="All Genes")
fig_ketamine.scatter(x="Ketamine log10FC", y="Ketamine padj", source=previous_source, fill_color="red", line_color="red", size=8, legend="Previous Genes")

for fig in figures:
    fig.renderers.extend([vline,hline])
    fig.legend.click_policy = "hide"
    
grid = gridplot([figures], toolbar_location="right")

show(grid)


##### Conclusion:
Most of the genes chosen previously to study were determined to be **statistically insignificant**. These are:

\* 0.05 

** 0.01

| Synonym | Gene | Propofol | Ketamine |
|---------|------|----------|----------|
| VK055_2084 | - | NS | NS |
| VK055_1930 | *fepC* | NS | NS |
| VK055_1398 | - | NS | NS |
| VK055_3638 | - | NS | NS |
| VK055_0094 | *copA* | NS | NS |
| VK055_4623 | *glnB* | NS | ** |
| VK055_1993 | - | NS | *  |
| VK055_4268 | - | NS | NS |
| VK055_3462 | - | ** | NS |
| VK055_3202 | *ilvC* | * | * |
| VK055_3875 | *yrbC* | ** | ** |
| 



 

### Scatter plot showing candidate genes fold-change in significance in experimental conditions
<img src="2019-01-10_f1_Draft3.png" width=800>

##### The genes marked by the legend were saved in tables

* [Only down in Propofol](2019-02-01_propofol_genes.csv)
* [Only down in Ketamine-Xylazine](2019-02-01_ketamine_genes.csv)
* [Down both](2019-02-01_both_genes.csv)

##### The scatter plot is interactive

    

In [ ]:
# Data sources
only_ket = ColumnDataSource(pd.read_csv("2019-02-01_ketamine_genes.csv", sep="\t"))
both_df = ColumnDataSource(pd.read_csv("2019-02-01_both_genes.csv",sep='\t'))
only_prop = ColumnDataSource(pd.read_csv("2019-02-01_propofol_genes.csv", sep="\t"))

# Initiate figures
fig_scatter = figure(plot_height=600, plot_width=600, title="Differentially Abundant Genes", 
            tools="crosshair,pan,reset, save,wheel_zoom, lasso_select, hover", tooltips=[
    ("Propofol P-Value adj", "@{Propofol padj}"),
    ("Ketamine P-Value adj", "@{Ketamine padj}"),
    ("Synonym","@Synonym"),
    ("Gene_ID", "@Gene"),
    ("Propofol Fold-Change", "@{Propofol log10FC}"),
    ("Ketamine Fold-Change", "@{Ketamine log10FC}")],
    x_axis_label='Propofol log10FC', y_axis_label='Ketamine log10FC')

fig_scatter.scatter(x="Propofol log10FC", y="Ketamine log10FC", source=only_ket,
                    fill_color="green", line_color="green", legend="Only in Propofol")
fig_scatter.scatter(x="Propofol log10FC", y="Ketamine log10FC", source=only_prop,
                    fill_color="blue", line_color="blue", legend="Only in Ketamine/Xylazine")
fig_scatter.scatter(x="Propofol log10FC", y="Ketamine log10FC", source=both_df,
                    fill_color="#e69f00", line_color="#e69f00", legend="Down in Both")

fig_scatter.legend.click_policy = "hide"

In [ ]:
show(fig_scatter)

### Possible Genes for studying further

After revising genes that were statistically significant in this INSeq experiment, I searched for genes that were essential for the growth of Klebsiella and virulence.

From this paper [Genome-Wide Identification of Klebsiella pneumoniae Fitness Genes during Lung Infection](https://mbio.asm.org/content/6/3/e00775-15.abstract), I selected a couple of genes and checked their Fold-Change under Propofol and Ketamine. These were:

* rfaH - Transcriptional Activator, know for serum resistance
* ilv operon - synthesis of branched amino-acids
    *ilvC
    *ilvD
* VK055_4417 - MarR family protein
* aroE - shikimate dehydrogenase required for aromatic amino acid synthesis

From previously selected genes, this ones were significant in one or both conditions: 
* glnB - 
* VK055_1993
* VK055_3462
* yrbC

In [ ]:
synonym_int = ["VK055_4417", "VK055_1993", "VK055_3462"]
gene_int = ['rfaH', 'ilvC', 'ilvD', 'glnB','yrbC', 'aroE']

synonym_df = deseq_df.set_index("Synonym")
gene_df = deseq_df.set_index("Gene")

found_df = pd.DataFrame()
for gene in gene_int:
    f = gene_df.loc[gene]
    found_df = found_df.append(f)
found_df = found_df.reset_index().set_index("Synonym")

for syn in synonym_int:
    f = synonym_df.loc[syn]
    found_df = found_df.append(f)
found_df = found_df.drop("Gene", axis=1).rename(columns={"index":"Gene"})
found_df

### Box plot

In [ ]:
fig_genes = plt.figure(constrained_layout=True, figsize=(10,8))
grid_gene = mpl.gridspec.GridSpec(3,3, figure=fig_genes)
r,c = 0,0
for i,row in found_df.iterrows():

    ax = fig_genes.add_subplot(grid_gene[r,c])
    if c == 2:
        r += 1
        c = 0
    else:
        c += 1
    gene = {
        "Input": list(row.iloc[10:13]),
        "Propofol": list(row.iloc[16:]),
        "Ketamine": list(row.iloc[13:16])
    }
    
    ax.boxplot(list(gene.values()), labels=list(gene.keys()), whis=10)
    if str(row.Gene) == "nan":
        ax.set_title(f"- ({row.name})")
    else:
        ax.set_title(f"{row.Gene} ({row.name})")
    

#### Select Genes Tool
* Use the buttons below to filter genes below the desired Fold-Change (default=0.25) and pvalue (default=0.05)

In [ ]:
@interact
def filter_df(condition=['Ketamine', 'Propofol', 'Both'], Fold_Change=widgets.FloatSlider(min=0,max=10,step=0.1, value=1/4) ,
              pvalue=widgets.FloatSlider(min=0,max=0.1,step=0.01, value=0.05)):
    
    if condition == "Both":
        output = deseq_df[ (deseq_df["Propofol" + ' log10FC'] < np.log10(Fold_Change)) & (deseq_df["Propofol" + " padj"] < pvalue)
                         & (deseq_df["Ketamine" + ' log10FC'] < np.log10(Fold_Change)) & (deseq_df["Ketamine" + " padj"] < pvalue)]
    elif condition != "Both":
        output = deseq_df[ (deseq_df[condition + ' log10FC'] < np.log10(Fold_Change)) & (deseq_df[condition + " padj"] < pvalue)]
    
    button = widgets.Button(description="Save genes")
    

    text_box = widgets.Textarea(
        value='',
        placeholder='Filename for saving genes',
        description='Output name:',
        disabled=False
        )

    display(button, text_box)
    def save_genes(b, text_box=text_box):
        output.to_csv(text_box.get_interact_value() + ".csv", sep='\t')
        return
    
    button.on_click(save_genes)
    return output
